<a href="https://colab.research.google.com/github/francji1/01NAEX/blob/main/code/01NAEX_Exercise_01_solution_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01NAEX - Exercise 01
Data and exercises come from D.C. Montgomery: Design and Analysis of Experiment


Get requirements:


In [ ]:
# get requirements for Lecture 1
list_of_packages <- c("tidyverse","nortest","lattice","pwr","MASS")
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)


In [ ]:
# If you need to check your settings

#getwd()
#print(.libPaths())
#print(sessionInfo())
#print(version)

## Assigment:

* Run and familiarize with R.
* Solve problems 2.20, 2.26, 2.30 from following slides.
  (originally from Montgomery - Design and Analysis of Experiments).


### Exercises 2.20

The shelf life of a carbonated beverage is of interest. Ten bottles are randomly
selected and tested, and the following results are obtained:

| Days ||
|--------------------------------------------||
| 108      |  138  |
| 124      |  163  |
| 124      |  159  |
| 106      |  134  |
| 115      |  139  |

* We would like to demonstrate that the mean shelf life exceeds 120 days.
Set up appropriate hypotheses for investigating this claim.
* Test these hypotheses using significant level $\alpha = 0.01$. Find the P-value
for the test. What are your conclusions?
* Construct a 99 percent confidence interval on the mean shelf life.

In [ ]:
Problem20 <- read.table("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex02_20.csv",header=TRUE,sep=";")
head(Problem20)


SOLUTION:

In [ ]:
t.test(Problem20,mu = 120,alternative = "less",conf.level = 0.99)

The null hypothesis $H_0: \mu > 120$ is tested with one sample t-test. As we can see in the summary, p-value exceeds the significance level and therefore the null hypothesis is **not rejected**. The 99% confidence interval is $\left(-\infty,148.4\right)$.


### Exercise 2.26

The following are the burning times (in minutes) of chemical flares of two different formulations. The design engineers are interested in both the mean and
variance of the burning times.

|Type1|| Type2 ||
|--------------------------------||||
| 65 | 82 | 64 | 56 |
| 81 | 67 | 71 | 69 |
| 57 | 59 | 83 | 74 |
| 66 | 75 | 59 | 82 |
| 82 | 70 | 65 | 79 |


1. Test the hypothesis that the two variances are equal. Use $\alpha = 0.05$.
2. Using the results of part 1), test the hypothesis that the mean burning
times are equal. Use $\alpha = 0.05$. What is the P-value for this test?
3. Discuss the role of the normality assumption in this problem. Check the
assumption of normality for both types of flares

In [ ]:
Problem26 <- read.table("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex02_26.csv",header=TRUE,sep=";")
head(Problem26)


SOLUTION:

**F-test vs Levene test**

The F-test (i.e., var.test()) assumes that both groups are normally distributed. If this assumption is not met, the F-test can give misleading results. Levene's test, on the other hand, is more robust against departures from normality, especially when centered at the median.

**Why use Levene test:** Given the sensitivity of the F-test to the normality assumption, in many cases (especially when normality is in question), the Levene's test might be preferred.

**Why use F-test:** The F-test can be more sensitive (i.e., powerful) when its assumptions are met. This means that if the data truly are normal, the F-test might be more likely to detect a difference in variances than the Levene's test.

In [ ]:
# F-test for variances
var.test_result <- var.test(Problem26$Type1, Problem26$Type2, alternative = "two.sided")
var.test_result

install.packages("lawstat")
library(lawstat)

long_data26 <- pivot_longer(Problem26, cols = everything(), names_to = "Type", values_to = "BurningTime")
values <- long_data26$BurningTime
groups <- long_data26$Type
levene_result <- levene.test(values, groups, location="median")
levene_result

With the results we obtained from F-test, we can conclude that variances of both types of substances are equal. Therefore we set option *var.equal = TRUE* in the following section, where we use two sample t-test.


In [ ]:
t.test(Problem26$Type1, Problem26$Type2, paired=FALSE, var.equal = TRUE,
       mu = 0,alternative = "two.sided",conf.level = 0.95)

*In* accordance with the statistical test, we do not reject the null hypothesis.

In [ ]:
# Shapiro-Wilk test for normality
shapiro.test_result_Type1 <- shapiro.test(Problem26$Type1)
shapiro.test_result_Type2 <- shapiro.test(Problem26$Type2)

shapiro.test_result_Type1
shapiro.test_result_Type2

# QQ-plots for visual inspection
par(mfrow=c(1, 2))
qqnorm(Problem26$Type1); qqline(Problem26$Type1, col="red")
qqnorm(Problem26$Type2); qqline(Problem26$Type2, col="red")

The assumption of normality is needed for both t-test and F-test. Using Shapiro-Wilk normality test we did not reject normality in either of the two samples.

### Exercise 2.30

Front housings for cell phones are manufactured in an injection molding process. The time the part is allowed to cool in the mold before removal is thought to influence the occurrence of a particularly troublesome cosmetic defect, flow lines, in the finished housing. After manufacturing, the housings are inspected visually and assigned a score between 1 and 10 based on their appearance, with 10 corresponding to a perfect part and 1 corresponding to a completely defective part. An experiment was conducted using two cool-down times, 10 and 20 seconds, and 20 housings were evaluated at each level of cool-down time. All 40 observations in this experiment were run in random order.


| 10s |||| 20s ||||
|--------------------------------||||||||
| 1 | 3 | 2 | 6 | 7 | 6 | 8 | 9 |
| 1 | 5 | 3 | 3 | 5 | 5 | 9 | 7 |
| 5 | 2 | 1 | 1 | 5 | 4 | 8 | 6 |
| 5 | 6 | 2 | 8 | 6 | 8 | 4 | 5 |
| 3 | 2 | 5 | 3 | 6 | 8 | 7 | 7 |


* Is there evidence to support the claim that the longer cool-down time
results in fewer appearance defects? Use $\alpha = 0.05$.
* What is the P-value for the test conducted in the previous part?
* Find a 95 percent confidence interval on the difference in means. Provide
a practical interpretation of this interval.
* Draw dot diagrams to assist in interpreting the results from this
experiment.


In [ ]:
Problem30 <- read.table("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex02_30.csv",header=TRUE,sep=";")
head(Problem30)

SOLUTION:

In [ ]:
var.test(Problem30$X10.seconds, Problem30$X20.seconds, alternative = "two.sided")

In [ ]:
# Convert the data into a long format zand use Levene test
long_data <- pivot_longer(Problem30, cols = everything(), names_to = "cooldown", values_to = "score")

values <- long_data$score
groups <- long_data$cooldown
levene.test(values, groups, location="median")

With F-test results, we can set *var.equal = TRUE*.

In [ ]:
t.test(Problem30$X10.seconds, Problem30$X20.seconds, paired=FALSE, var.equal = TRUE,
       mu = 0,alternative = "greater",conf.level = 0.95)

In [ ]:
diff_means = mean(Problem30$X10.seconds) - mean(Problem30$X20.seconds)
print("Difference in means:")
print(diff_means)

With p-value = 1, we do not reject hypothesis. The confidence interval is $\left(-4.1 , +\infty\right)$. There is a 95% statistical confidence that the real difference in means falls into this interval. The difference in sample means equals -3.15 which is inside of the confidence interval.

In [ ]:
Problem30 %>%
  pivot_longer(cols = c(X10.seconds,X20.seconds), names_to = "type", values_to = "values") %>%
  ggplot(aes(x=type, y=values, fill = type)) +
  geom_dotplot(binaxis='y', stackdir='center',
                stackratio=1.5, dotsize=1.2,
                binwidth = 0.3) +
  scale_y_discrete(limits=factor(1:9)) +
  scale_x_discrete(labels = c('10s','20s')) +
  labs(title="Dot diagram",
         x ="Time of cool down", y = "Quality") +
  coord_flip() + theme(legend.position="none")

In [ ]:
# simmilarway
long_data <- gather(Problem30, key="cooldown", value="score")
ggplot(long_data, aes(x=cooldown, y=score, fill=cooldown)) +
  geom_dotplot(binaxis="y", stackdir="center", dotsize=1, binwidth=0.5) +
  labs(title="Appearance Scores for Different Cool-Down Times", y="Appearance Score", x="Cool-Down Time") +
  theme_minimal() +
  scale_fill_discrete(name="Cooldown Time")